### Using the [PyPortfolioOpt library](https://github.com/robertmartin8/PyPortfolioOpt) library to generate an optimzied portfolio from the S&P 500 list of companies
Reference: [Build A Killer Stock Portfolio Using Python](https://www.youtube.com/watch?v=bvDkel5whUY&t=2s&ab_channel=ComputerScience)

<b><font color="red">Click <a href="https://colab.research.google.com/github/ebharucha/Portfolio-Optimization/blob/master/PortfolioOpt.ipynb" target="#">here</a> to open/run the notebook in Google Colab</font></b>

@ebharucha 12/31/2020

### Install & import dependencies

In [216]:
!pip install PyPortfolioOpt
!pip install pulp

In [220]:
import pandas as pd
import pandas_datareader as web
import datetime
import pickle
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import risk_models, expected_returns

### Get & initialize data

#### <font color="Red">Specify value of overall portfolio in USD</font>

In [221]:
portfolio_val = 50000

#### Get S&P 500 companies

In [183]:
DATADIR = './data'

table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_SP500_ = table[0]
df_SP500_.Symbol = df_SP500_.Symbol.replace('\.','-', regex=True)
df_SP500_.to_csv(f'{DATADIR}/S&P500-Info.csv')
df_SP500_.to_csv(f'{DATADIR}/S&P500-Symbols.csv', columns=['Symbol'])

#### Load daily closing prices for S&P 500 components over past 10 years in a DataFrame

<i><font color="Red">Only run this once per day to get updates.
    If already run earlier in the day, then load data from pickle file</font></i>

In [153]:
symbols = df_SP500_.Symbol
df_SP500 = pd.DataFrame(columns=symbols)

current_year = datetime.datetime.now().year
today = str(datetime.date.today())

f = lambda sym: web.get_data_yahoo(sym,
                            start = (f'{current_year-10}-01-01'),
                            end = today)['Adj Close']

for sym in symbols:
    df_SP500[sym] = f(sym)

#### Pickle the data

In [188]:
# with open (f'{DATADIR}/SP500.pkl', 'wb') as pklfile:
#     pickle.dump(df_SP500, pklfile)

with open(f'{DATADIR}/SP500.pkl', 'rb') as pklfile:
    df_SP500 = pickle.load(pklfile)

#### Get latest prices

In [227]:
latest_prices = get_latest_prices(df_SP500)

### Portfolio optimization

#### Expected annualized returns & annualized covariance matrix of the daily asset returns

In [202]:
mu = expected_returns.mean_historical_return(df_SP500)
S = risk_models.sample_cov(df_SP500)

C:\Users\erb\AppData\Local\Continuum\anaconda3\lib\site-packages\pypfopt\risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
C:\Users\erb\AppData\Local\Continuum\anaconda3\lib\site-packages\pypfopt\risk_models.py:88: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn("Could not fix matrix. Please try a different risk model.")


#### Optimzie for the maximal Sharpe ratio

In [222]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
weights = cleaned_weights
# print (cleaned_weights)
ef.portfolio_performance(verbose=True)

Expected annual return: 141.0%
Annual volatility: 28.7%
Sharpe Ratio: 4.84


(1.4101475726538226, 0.2871356355576669, 4.841431715551141)

#### Get stock allocations

In [230]:
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=portfolio_val)
allocation, leftover = da.lp_portfolio()
print (f'Discrete share allocations: {allocation}')
print (f'Funds remaining: ${leftover:.2f}')

Discrete share allocations: {'CARR': 413, 'CHTR': 3, 'CLX': 19, 'DG': 7, 'DPZ': 26, 'LUMN': 1, 'NFLX': 4, 'POOL': 8, 'FTI': 1, 'VNT': 325, 'WMT': 8, 'WEC': 3}
Funds remaining: $7.12


In [284]:
# Get company name & sector
name = lambda sym: df_SP500_[df_SP500_.Symbol == sym].Security.values[0]
sector_ = lambda sym: df_SP500_[df_SP500_.Symbol == sym]['GICS Sector'].values[0]

name('NFLX'), sector_('NFLX')

('Netflix Inc.', 'Communication Services')

#### Construct portfolio DataFrame

In [298]:
port_df = pd.DataFrame(columns=['Symbol', 'Company Name', "Sector", "No. of Shares", "Allocation($)"])

In [305]:
symbols = allocation.keys()
company_name = []
sector = []
no_of_shares = []
allocation_dollar = []
total_allocation = 0

for sym in symbols:
    company_name.append(name(sym))
    sector.append(sector_(sym))
    no_of_shares.append(allocation.get(sym))
    allocation_dollar.append(f'{allocation.get(sym) * latest_prices[sym]:,.2f}')
    total_allocation = total_allocation + allocation.get(sym) * latest_prices[sym]

In [303]:
port_df['Symbol'] = symbols
port_df['Company Name'] = company_name
port_df['Sector'] = sector
port_df['No. of Shares'] = no_of_shares
port_df['Allocation($)'] = allocation_dollar

### <font color="red">Display portfolio allocations & expected performance</dont>

In [326]:
port_df

,Symbol,Company Name,Sector,No. of Shares,Allocation($)
0,CARR,Carrier Global,Industrials,413,"15,582.49"
1,CHTR,Charter Communications,Communication Services,3,"1,960.47"
2,CLX,The Clorox Company,Consumer Staples,19,"3,819.76"
3,DG,Dollar General,Consumer Discretionary,7,"1,469.51"
4,DPZ,Domino's Pizza,Consumer Discretionary,26,"10,000.12"
5,LUMN,Lumen Technologies,Communication Services,1,9.72
6,NFLX,Netflix Inc.,Communication Services,4,"2,098.36"
7,POOL,Pool Corporation,Consumer Discretionary,8,"2,938.24"
8,FTI,TechnipFMC,Energy,1,9.62
9,VNT,Vontier,Information Technology,325,"10,679.50"


In [327]:
print (f'Total allocated amount = ${total_allocation:,.2f}')
print (f'Amount remaining: ${leftover:,.2f}')
ef.portfolio_performance(verbose=True)

Total allocated amount = $49,992.88
Amount remaining: $7.12
Expected annual return: 141.0%
Annual volatility: 28.7%
Sharpe Ratio: 4.84


(1.4101475726538226, 0.2871356355576669, 4.841431715551141)